# Como instalar e configurar o Hadoop no CentOS parte 1/2

Fala pessoal, tudo na paz?

Neste artigo vou demonstrar como instalar e configurar o Hadoop no sistema operacional CentOS que está rodando sobre uma VM no VirtualBox.

### Explicar sobre o Hadoop

## Instalação do Hadoop

### Utilitários

Primeiramente irei instalar alguns pacotes utilitários, pois se você acompanhou as últimas postagens sobre o como instalar Linux em uma VM viu que eu instalei apenas o básico para o funcionamento do SO com interface gráfica. Então vamos lá, no terminal:

> `sudo yum install bzip2 unzip rsync wget net-tools`

### MySQL (MariaDB)

É recomendado ter um banco de dados no SO. Portanto irei instalar o mais simples deles, o MariaDB, que é uma versão do MySQL. No terminal:

> `sudo yum install mariadb-server mariadb`

Com a instalação concluída vamos inicializar o serviço de banco de dados e habilitar a sua inicialização no boot com os comandos:

> `sudo systemctl start mariadb.service`<br/>
> `sudo systemctl enable mariadb.service`<br/>

Definindo a senha para o usuário root do Banco de Dados:

> `mysqladmin -u root passwrod 'sua_senha'`

Vamos testar a instalação iniciando o console do MySQL:

> `mysql -u root -p` <br/>
> Digite a sua senha:

Se tudo deu certo é esta a tela que está aparecendo para você no terminal:

<img src="resources/00.png">

Agora que estamos no console do MariaDB, vamos testar um comando simples só para garantir que tudo está funcionando corretamente:

> `SELECT user, host, password FROM mysql.user;`

> Nota: Não esqueça de utilizar o "; (ponto e vírgula)" no final das instruções SQL aqui no terminal.

<img src="resources/01.png">

Para sair do console do MariaDB digite `exit;`

### Instalação do servidor SSH

SSH é a sigla para Secure Socket Shell, sendo um dos protocolos específicos de segurança de troca de arquivos entre cliente e servidor de internet, usando criptografia. O objetivo do SSH é permitir que desenvolvedores ou outros usuários realizem alterações em sites e servidores utilizando uma conexão simples e segura.

No terminal:

> `sudo yum install openssh-server openssh-clients`

Ativando o servidor SSH no boot de inicialização:

> `sudo chkconfig sshd on`

Inicializando o serviço:

> `sudo service sshd start`

Para verificar se deu certo:

> `sudo service sshd status`

<img src="resources/02.png">

Outra forma de averiguar é verificando as informações de rede na porta 22:

> `sudo netstat -tulpn | grep :22`

<img src="resources/03.png">

Agora daremos permissão para que o usuário root possa acessar livremente na porta 22. No terminal:

> `sudo gedit /etc/ssh/sshd_config`

No documento que foi abert, remova o # no início das linhas selecionadas:

Primeira parte:<br/>

<img src="resources/04.png">

Segunda Parte:<br/>
> Adicione o seu usuário.<br/>

<img src="resources/05.png">

Reinicializando o serviço ssh:

> `sudo service sshd restart`

### Instalação do JRE (Java Runtime Edition)

Como o Hadoop roda sobre a linguagem de programação Java, precisaremos instalar tanto o JRE quanto JDK na nossa máquina:

> `sudo yum install java`

### Instalação do JDK (Java Development Kit)

A mesma coisa para o JDK, mas o processo para instalação do JDK é um pouco diferente. Será necessário navegar até o site da Oracle. Você pode acessar clicando nesse link: <a target="_blank" href="https://www.oracle.com/java/technologies/downloads/">Oracle.com</a>

<img src="resources/06.png">

Navegando pelo diretório do arquivo e listando conteúdo da pasta Downloads:

<img src="resources/07.png">

Agora precisamos descompactar o arquivo e mover ele para outro local:

> `tar -xzf jdk-8u202-linux-x64.tar.gz`

Irei mover o jdk que foi extraído para o diretório /usr/local/hadoop_dependencies. Os diretórios usr/ e local/ já existem por padrão. Para criar o diretório hadoop_dependencies, vá em:
> `cd /usr/local` -> Vá até o diretório usr/local/<br/>
> `sudo mkdir hadoop_dependencies` Crie o diretório hadoop_dependencies/<br/>
> `cd ~` Volte para o diretório raiz<br/>
> `cd Downloads/` Navegue para o diretório Downloads/<br/>

Vamos utilizar esse diretório para armazenar todos os pacotes que baixarmos durante a nossa instalação do Hadoop. Dessa forma os arquivos ficam todos organizados.

> `sudo mv jdk1.8.0_202/ /usr/local/hadoop_dependencies/`

Agora iremos criar um link simbólico. Entenda o link simbólico como um atalho para algum diretório ou arquivo. Portanto, quando nos referimos ao java utilizaremos o link simbólico. Dessa forma, é possível controlar as versões dos pacotes de maneira muito mais dinâmica e profissional. Pois não teremos que alterar dezenas de arquivos todas as vezes que alterarmos a versão de algum pacote.

> `sudo ln -s /usr/local/hadoop_dependencies/jdk1.8.0_202/ /opt/jdk`

Faremos a mesma coisa para o JRE:

> `sudo ln -s /usr/lib/jvm/java-1.8.0-openjdk-1.8.0.342.b07-1.el7_9.x86_64/jre/ /opt/jre`

Verificando os links simbólicos:

<img src="resources/08.png">

Perceba que o prefixo do tipo do objeto é `l` que significa link.

**Adicionando variáveis de ambiente**

Agora que instalamos o JRE e o JDK, precisamos definir as variáveis de ambiente para que o SO possa identifcar os pacotes que foram instalados.

> `cd ~`<br/>
> `gedit .bashrc`<br/>

O arquivo .bashrc é o arquivo que armazena as variáveis de ambiente. Dentro desse arquivo adicione os valores:

> export JAVA_HOME=/opt/jdk <br/>
> export JRE_HOME=/opt/jre <br/>
> export PATH=$PATH:$JRE_HOME/bin:$JAVA_HOME/bin <br/>

<img src="resources/09.png">

Para atualizar as novas configurações, digite:

> `source .bashrc`

### Desabilitando o IPv6

Alguns recursos do Hadoop não funcionam muito bem com o este protocólo. Portanto, iremos desabilitar para evitar qualquer problema futuro. No terminal, vá em:

> `sudo gedit /etc/sysctl.conf`<br/>

No arquivo de configuração, adicione as seguintes configurações:

> `net.ipv6.all.disable_ipv6 = 1`<br/>
> `net.ipv6.default.disable_ipv6 = 1`<br/>
> `net.ipv6.lo.disable_ipv6 = 1`<br/>




### Configurando o SSH para o Hadoop

Anteriormente havia ativado o SSH, agora nós vamos configurar o SSH para o Hadoop. A primeira coisa que vamos fazer é gerar uma chave de segurança. O SSH é um protocólo de comunicação. No terminal:

> `ssh-keygen -t rsa`

Nesse momento, o SO vai perguntar aonde você quer criar essa chave. Basta digitar enter para criar no diretório sugerido. 
<img src="resources/10.png">

Agora, ele vai perguntar pedir para você digitar uma passphrase. É como se fosse uma senha. Não precisa digitar nada aqui, aperte Enter duas vezes e pronto.

<img src="resources/11.png">

Precisamos que esta chave gerada seja reconhecida e garantida pelo SSH. Portanto, devemos copiar a chave para um arquivo de chaves autorizadas:

> `cat ~/.ssh/id_rsa.pub >> ~/.ssh/authorized_keys` Copiando o arquivo. <br/>
> `chmod 0600 ~/.ssh/authorized_keys` Definindo permissões para esse arquivo. <br/>

Podemos testar o ssh digitando `ssh localhost` no terminal. Digite yes e pronto, conexão estabelecidade.

**Mas para que serve isso afinal?**<br/>
Em um ambiente de cluster real, esse protocólo vai permitir que o NameNode se comunique com o DataNode através do protocólo SSH e sem a necessidade de senhas. Porque imagine só um cluster com 1000 máquinas, onde cada máquina precisa digitar a senha para se comunicar com o NameNode? Seria inviável, por isso configuramos o SSH.

Para finalizar a configuração do SSH, vamos dizer quais são as máquinas que podem conectar no nosso servidor.

> `sudo gedit /etc/hosts`

<img src="resources/12.png">

---

### Instalação do Hadoop

Agora sim estamos prontos para começar a instalar o Hadoop. A instalação do hadoop em si é bastante simples, porém, realizar todas as configurações necessárias para trabalharmos com um cluster, aí sim teremos um pouco mais de trabalho.

Acesso o site oficial para downloads do Hadoop: [download dos binários](https://dlcdn.apache.org/hadoop/common/hadoop-3.3.4/hadoop-3.3.4.tar.gz)

Por padrão o arquivo será baixado no diretório de Downloads. Mas você pode baixar totalmente via linha de comando no diretório atual inserindo:

> `wget https://dlcdn.apache.org/hadoop/common/hadoop-3.3.4/hadoop-3.3.4.tar.gz`

O arquivo foi baixado com sucesso, veja:

<img src="resources/13.png">

Descompactando o arquivo:

> `tar -xzf hadoop-3.3.4.tar.gz`

Faremos um passo a passo bem parecido com a instalação do JDK. Primeiro movemos o arquivo para o diretório /usr/local/hadoop_dependencies, depois criamos um links simbólico e configuramos as variáveis de ambiente.

> `sudo mv hadoop-3.3.4 /usr/local/hadoop_dependencies/`Movendo o arquivo para outro diretório<br/>
> `sudo ln -s /usr/local/hadoop_dependencies/hadoop-3.3.4/ /opt/hadoop` Criação do link simbólico<br/>

No diretório raiz:
> `gedit .bashrc`

<img src="resources/20.png">

> `source .bashrc`

---

### Adicionando parâmetros de Configuração

Temos o Hadoop instalado. Precisamos realizar as configurações em seus arquivos de configurações. Navegue até o diretório de arquivos de configuração do Hadoop:

> `cd /opt/hadoop/etc/hadoop`

<img src="resources/14.png">

Será necessário editar cinco arquivos neste diretório, são eles:


> `gedit hadoop-env.sh`

Adicione os seguintes valores:

<img src="resources/15.png">

---

> `gedit core-site.xml`

Veja que o arquivo vem vazio, porque o Hadoop está no modo Standalone. Queremos colocá-lo em modo Pseudo-Distribuído. Para isso, devemos adicionar tags de configuração:

<img src="resources/16.png">

Esse parâmetro indica que o sistema de arquivos padrão para o meu Hadoop será o HDFS e que tudo será apontado para o localhost na porta 9000.

---

> `gedit hdfs-site.xml`

Esse parâmetro dfs.replication com o valor igual a 1 basicamente diz que não queremos replicar os blocos de arquivos para mais de um node. 

<img src="resources/17.png">

---

> `gedit mapred-site.xml`

Aqui eu vou informar para o Hadoop como eu quero que seja feita a gestão dos recursos. E essa gestão será feita pelo Yarn.

<img src="resources/18.png">

---

> `gedit yarn-site.xml`

Um dos serviços que queremos que o Yarn gerencie é o mapreduce shuffle, etapa intermediária entre o map e o reduce.

<img src="resources/19.png">

---

### Vericando instalação

Depois de todas essa configurações vamos finalmever averiguar se o Hadoop está funcionando:

> `hadoop version`

<img src="resources/21.png">

Esta é a mensagem que deve aparecer para você caso o Hadoop tenha sido instalado corretamente.

---

Verificando se os arquivos de configuração foram instalados corretamente:

> `hdfs namenode -format`

<img src="resources/22.png">

Aqui a mensagem que queremos, indicando se ocorreu tudo bem com a inicialização do namenode.


---

#### 02.09.05 Instalação do Haddop p5
#### PG: 135/266